## Atividade 3

#### Iniciar sessão Spark


In [10]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum

spark = SparkSession.builder \
    .appName("Atividade3") \
    .getOrCreate()


#### Carregar ambos os arquivos CSV 


In [11]:
df_clientes = spark.read.csv("clientes.csv", header=True, inferSchema=True)
df_transacoes = spark.read.csv("transacoes.csv", header=True, inferSchema=True)

df_clientes.show()
df_transacoes.show()

+----------+-----+-----+--------------+
|id_cliente| nome|idade|        cidade|
+----------+-----+-----+--------------+
|         1| João|   35|     São Paulo|
|         2|Maria|   25|Rio de Janeiro|
|         3| José|   40|      Curitiba|
|         4|  Ana|   28|     São Paulo|
|         5|Paulo|   50|Belo Horizonte|
+----------+-----+-----+--------------+

+------------+----------+-----+--------------+-----------+
|id_transacao|id_cliente|valor|data_transacao|  categoria|
+------------+----------+-----+--------------+-----------+
|           1|         1|100.0|    2023-11-01|    Compras|
|           2|         2|200.0|    2023-11-02|Alimentação|
|           3|         3|150.0|    2023-11-03|    Compras|
|           4|         4|300.0|    2023-11-01|     Viagem|
|           5|         5| 50.0|    2023-11-04|Alimentação|
+------------+----------+-----+--------------+-----------+



#### Realizar o join entre os DataFrames

In [12]:
df_joined = df_transacoes.join(
    df_clientes,
    on="id_cliente",
    how="inner"
)

df_joined.show()


+----------+------------+-----+--------------+-----------+-----+-----+--------------+
|id_cliente|id_transacao|valor|data_transacao|  categoria| nome|idade|        cidade|
+----------+------------+-----+--------------+-----------+-----+-----+--------------+
|         1|           1|100.0|    2023-11-01|    Compras| João|   35|     São Paulo|
|         2|           2|200.0|    2023-11-02|Alimentação|Maria|   25|Rio de Janeiro|
|         3|           3|150.0|    2023-11-03|    Compras| José|   40|      Curitiba|
|         4|           4|300.0|    2023-11-01|     Viagem|  Ana|   28|     São Paulo|
|         5|           5| 50.0|    2023-11-04|Alimentação|Paulo|   50|Belo Horizonte|
+----------+------------+-----+--------------+-----------+-----+-----+--------------+



#### Calcular o valor total de transações por cliente


In [13]:
df_total_transacoes = df_joined.groupBy("id_cliente", "nome") \
    .agg(sum("valor").alias("total_transacoes"))

df_total_transacoes.show()

+----------+-----+----------------+
|id_cliente| nome|total_transacoes|
+----------+-----+----------------+
|         3| José|           150.0|
|         5|Paulo|            50.0|
|         4|  Ana|           300.0|
|         1| João|           100.0|
|         2|Maria|           200.0|
+----------+-----+----------------+



#### Identificar o cliente com o maior volume

In [14]:
cliente_maior_volume = df_total_transacoes.orderBy(col("total_transacoes").desc()).first()

print(f"""
Cliente com maior volume: 
Nome: {cliente_maior_volume['nome']}
ID: {cliente_maior_volume['id_cliente']}
Total: R${cliente_maior_volume['total_transacoes']:.2f}
""")


Cliente com maior volume: 
Nome: Ana
ID: 4
Total: R$300.00

